<a href="https://colab.research.google.com/github/wspalding/cozzez/blob/master/News_statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Collect News Stories

In [0]:
%%capture
!pip install mechanicalsoup

In [0]:
import mechanicalsoup as msoup
import datetime
browser = msoup.StatefulBrowser()

## MSNBC

In [12]:
browser.open("https://www.msnbc.com/")
page = browser.get_current_page()
urls = []
#print(len(articles))
#msnbc.com website structure
#all -> div.id = content
#top story = section2 -> a.href
#seconday story = section3 -> article -> a.href
#third set story = section4 -> article -> a.href
#fourth & fifth = section5 -> a.href
#sixth = section 6 -> ul -> li -> h2 -> a.href
#vids = section7 -> video
content = page.find_all('div', {'id': 'content'})
#urls.append(len(content))
for cont in content:
    #secs = cont.find_all('section')
    #url.append(secs)
    for sec in cont.find_all('section'):
        for a in sec.find_all('a'):
            urls.append(a.attrs.get('href', None))
        for ul in sec.find_all('ul'):
            for li in ul.find_all('li'):
                for h2 in li.find_all('h2'):
                    for a in h2.find_all('a'):
                        urls.append(a.attrs.get('href', None))
        for h3 in sec.find_all('h3'):
            for a in h3.find_all('a'):
                urls.append(a.attrs.get('href', None))
        for article in sec.find_all('article'):
            for a in article.find_all('a'):
                urls.append(a.attrs.get('href', None))
    
    #print(url)
urls = set(i for i in urls if not any(i in s for s in urls if i != s))
urls = set(i for i in urls if (i.startswith("https://www.msnbc.com") or i.startswith("https://www.nbcnews.com")))

urls = [i for i in urls if i.count("/") >=5]
print("collected: ", len(urls), " urls")

collected:  112  urls


In [21]:
info = {}
for url in urls:
    try:
        response = browser.open(url)
    except:
        cur_dict["status_code"] = "Error could not open url"
        continue
    page = browser.get_current_page()
    info[url] = {}
    cur_dict = info[url]
    response = browser.open(url)
    cur_dict["date"] = datetime.date.today()
    cur_dict["source"] = "MSNBC"
    cur_dict["source_url"] = "https://www.msnbc.com/"
    cur_dict["status_code"] = response.status_code
    page = browser.get_current_page()
    if response.status_code != 200:
        continue;
    cur_dict["title"] = page.find('h1').text
    cur_dict["text"] = ""
    # cur_dict["labels"] = get_labels(cur_dict["text"], cur_dict["title"])
    cur_dict["author"] = "unknown"
    author = page.find('section', {'class':'mb7'})
    if author is None:
        index = url.find("www.msnbc.com/")
        end_index = url.find("/watch/")
        if index >= 0 and end_index > index:
            cur_dict["author"] = url[index+len("www.msnbc.com/"):end_index]
    if author is not None:
        index = author.text.find("By ")
        if index > 0:
            cur_dict["author"] = author.text[index+len("By "):]
        else:
            cur_dict["author"] = "unknown"
        #cur_dict["author"] = author.text
    cur_dict["media"] = None
    text = page.find_all('p')
    full_text = ""
    copy_right = "This material may not be"
    #if type(text) != list:
    #    text = [text]
    for t in text:
        if copy_right not in t:
            full_text += "\n" + t.text
    real_text = full_text.encode('ascii', 'ignore')
    #print(full_text)
    real_text = real_text.decode("utf-8")
    #real_text = real_text.replace(copy_right, "")
    #real_text = real_text.replace(copy_right2, "")
    cur_dict["text"] = real_text
    # cur_dict["labels"] = get_labels(real_text,
    #                                 cur_dict["title"])
# print(info)

{'https://www.msnbc.com/deadline-white-house/watch/questions-mount-about-whether-trump-is-actually-profiting-from-the-presidency-68910149949': {'date': datetime.date(2019, 9, 16), 'source': 'MSNBC', 'source_url': 'https://www.msnbc.com/', 'status_code': 200, 'title': 'Questions mount about whether Trump is actually profiting from the presidency', 'text': '\nNYTs Mike Schmidt, former U.S. attorney Joyce Vance, former chief spokesman for the Department of Justice Matt Miller, MoveOns Karine Jean-Pierre, former congressman David Jolly and host of Politics Nation Reverend Al Sharpton on the many recent stories involving government officials and Trump properties that raise questions about Donald Trumps conflicts of interestSept. 13, 2019\n\n2019 NBC UNIVERSAL', 'author': 'deadline-white-house', 'media': None}, 'https://www.nbcnews.com/news/world/twitter-mocks-boris-johnson-after-he-compares-himself-hulk-n1054606': {'date': datetime.date(2019, 9, 16), 'source': 'MSNBC', 'source_url': 'https: